# 05.1a: Identify Tokens by Direction

**Goal:** Point at a specific location on the sky map and identify all tokens within an angular radius.

Given:
- **Target longitude** (φ): position along the equator (-180° to +180°)
- **Target latitude** (θ): CDF-flattened coordinate from sky map (-90° to +90°)
- **Angular radius** (α): how wide the "telescope field of view" is

We'll:
1. Compute CDF-flattened spherical coordinates for all tokens
2. Convert target (lon, lat_CDF) to Cartesian unit vector in CDF-flattened space
3. Compute angular distance in CDF-flattened space for all tokens
4. Filter tokens within angular radius
5. Display results sorted by angular distance

## Parameters

In [54]:
TENSOR_DIR = "../data/tensors"
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"

# Target direction (read straight from 04.1a sky map)
TARGET_LON = 164.5     # degrees, -180 to +180
TARGET_LAT_CDF = 7.0   # degrees, -90 to +90 (CDF-flattened from sky map)

# Angular radius (field of view)
ANGULAR_RADIUS = 1.0  # degrees

# Coordinate system (must match 04.1a)
NORTH_PC = 1
MERIDIAN_PC = 2
EQUINOX_PC = 3

## Imports

In [55]:
import torch
import numpy as np
import pandas as pd
from safetensors.torch import load_file
from pathlib import Path
from transformers import AutoTokenizer

print("Imports loaded successfully.")

Imports loaded successfully.


## Step 1: Load Data

In [66]:
# Load centered gamma prime
gamma_prime_path = Path(TENSOR_DIR) / "gamma_centered_qwen3_4b_instruct_2507.safetensors"
gamma_prime = load_file(gamma_prime_path)['gamma_centered']

N, d = gamma_prime.shape

print(f"Loaded γ' (centered):")
print(f"  Tokens: {N:,}")
print(f"  Dimensions: {d:,}")
print()

Loaded γ' (centered):
  Tokens: 151,936
  Dimensions: 2,560



## Step 2: Compute PCA

In [67]:
print("Computing covariance matrix...")
Cov = (gamma_prime.T @ gamma_prime) / (N - 1)

print(f"Computing eigendecomposition...\n")
eigenvalues, eigenvectors = torch.linalg.eigh(Cov)

# Sort descending
eigenvalues = eigenvalues.flip(0)
eigenvectors = eigenvectors.flip(1)

# Extract basis vectors
north_vector = eigenvectors[:, NORTH_PC - 1]
meridian_vector = eigenvectors[:, MERIDIAN_PC - 1]
equinox_vector = eigenvectors[:, EQUINOX_PC - 1]

print(f"PCA computed. Using coordinate system:")
print(f"  North: PC{NORTH_PC}")
print(f"  Meridian: PC{MERIDIAN_PC}")
print(f"  Equinox: PC{EQUINOX_PC}")

Computing covariance matrix...
Computing eigendecomposition...

PCA computed. Using coordinate system:
  North: PC1
  Meridian: PC2
  Equinox: PC3


## Step 3: Compute Spherical Coordinates (CDF-Flattened)

We'll compute spherical coordinates in CDF-flattened space to match the sky map visualization.

In [68]:
print(f"Computing spherical coordinates for all tokens...\n")

# Project onto basis vectors
x = gamma_prime @ meridian_vector
y = gamma_prime @ equinox_vector
z = gamma_prime @ north_vector

# Compute radius
r = torch.sqrt(x**2 + y**2 + z**2)

# Original latitude
theta_rad = torch.asin(torch.clamp(z / r, -1, 1))
theta_deg = torch.rad2deg(theta_rad)

# Longitude (same in both coordinate systems)
phi_rad = torch.atan2(y, x)
phi_deg = torch.rad2deg(phi_rad)

# Compute CDF-flattened latitude
theta_sorted, sort_indices = torch.sort(theta_deg)
cdf_values = torch.linspace(0, 1, N)
theta_flat_sorted = 90.0 * (2.0 * cdf_values - 1.0)
theta_flat = torch.zeros_like(theta_deg)
theta_flat[sort_indices] = theta_flat_sorted

print(f"Spherical coordinates computed.")
print(f"  Latitude range (original): [{theta_deg.min().item():.2f}°, {theta_deg.max().item():.2f}°]")
print(f"  Latitude range (CDF-flat): [{theta_flat.min().item():.2f}°, {theta_flat.max().item():.2f}°]")
print(f"  Longitude range: [{phi_deg.min().item():.2f}°, {phi_deg.max().item():.2f}°]")

Computing spherical coordinates for all tokens...

Spherical coordinates computed.
  Latitude range (original): [-89.92°, 89.68°]
  Latitude range (CDF-flat): [-90.00°, 90.00°]
  Longitude range: [-179.99°, 180.00°]


## Step 4: Convert Target Direction to Cartesian (in CDF-Flattened Space)

In [69]:
print(f"Target direction (from sky map):") 
print(f"  Longitude: {TARGET_LON}°")
print(f"  Latitude (CDF-flattened): {TARGET_LAT_CDF}°")
print()

# Convert CDF-flattened spherical coords to Cartesian unit vector
lon_rad = np.deg2rad(TARGET_LON)
lat_cdf_rad = np.deg2rad(TARGET_LAT_CDF)

# Spherical to Cartesian (unit sphere in CDF-flattened space)
x_target = np.cos(lat_cdf_rad) * np.cos(lon_rad)
y_target = np.cos(lat_cdf_rad) * np.sin(lon_rad)
z_target = np.sin(lat_cdf_rad)

target_direction = torch.tensor([x_target, y_target, z_target], dtype=torch.float32)

print(f"Target direction (CDF-flattened Cartesian):")
print(f"  x: {x_target:.6f}")
print(f"  y: {y_target:.6f}")
print(f"  z: {z_target:.6f}")
print(f"  norm: {target_direction.norm().item():.6f} (should be 1.0)")

Target direction (from sky map):
  Longitude: 164.5°
  Latitude (CDF-flattened): 7.0°

Target direction (CDF-flattened Cartesian):
  x: -0.956448
  y: 0.265246
  z: 0.121869
  norm: 1.000000 (should be 1.0)


## Step 5: Compute Angular Distances (in CDF-Flattened Space)

For each token, convert its CDF-flattened (lon, lat_CDF) to a Cartesian unit vector, then compute angular distance to target:

$$\theta = \arccos(\vec{v}_{\text{token}} \cdot \vec{v}_{\text{target}})$$

In [70]:
print(f"Computing angular distances for all {N:,} tokens in CDF-flattened space...\n")

# Convert all tokens' CDF-flattened (lon, lat_CDF) to Cartesian unit vectors
phi_rad_all = torch.deg2rad(phi_deg)
theta_flat_rad = torch.deg2rad(theta_flat)

x_all = torch.cos(theta_flat_rad) * torch.cos(phi_rad_all)
y_all = torch.cos(theta_flat_rad) * torch.sin(phi_rad_all)
z_all = torch.sin(theta_flat_rad)

# Stack into (N, 3) matrix
token_directions = torch.stack([x_all, y_all, z_all], dim=1)

# Dot product with target direction (cosine of angular distance)
cosine_sims = token_directions @ target_direction

# Convert to angular distance
cosine_sims_clamped = torch.clamp(cosine_sims, -1.0, 1.0)
angular_distances_rad = torch.acos(cosine_sims_clamped)
angular_distances_deg = torch.rad2deg(angular_distances_rad)

print(f"Angular distance statistics:")
print(f"  Min: {angular_distances_deg.min().item():.4f}°")
print(f"  Max: {angular_distances_deg.max().item():.4f}°")
print(f"  Mean: {angular_distances_deg.mean().item():.4f}°")
print(f"  Median: {angular_distances_deg.median().item():.4f}°")

Computing angular distances for all 151,936 tokens in CDF-flattened space...

Angular distance statistics:
  Min: 0.1170°
  Max: 179.8287°
  Mean: 91.4744°
  Median: 94.2734°


## Step 6: Filter Tokens Within Angular Radius

In [71]:
# Find tokens within angular radius
within_radius = angular_distances_deg <= ANGULAR_RADIUS
n_within = within_radius.sum().item()

print(f"\nTokens within {ANGULAR_RADIUS}° of ({TARGET_LON}°, {TARGET_LAT_CDF}° CDF):")
print(f"  Count: {n_within:,}")
print(f"  Percentage: {n_within/N*100:.4f}%")

if n_within == 0:
    print("\nNo tokens found. Try increasing ANGULAR_RADIUS.")
elif n_within > 10000:
    print(f"\nWARNING: {n_within:,} tokens found! Large table ahead.")
    print(f"Consider reducing ANGULAR_RADIUS.")


Tokens within 1.0° of (164.5°, 7.0° CDF):
  Count: 111
  Percentage: 0.0731%


## Step 7: Load Tokenizer

In [72]:
if n_within > 0:
    print(f"\nLoading tokenizer: {MODEL_NAME}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    print(f"Tokenizer loaded.")


Loading tokenizer: Qwen/Qwen3-4B-Instruct-2507...
Tokenizer loaded.


## Step 8: Create Results Table

In [73]:
if n_within > 0:
    # Get indices of tokens within radius
    indices_within = torch.where(within_radius)[0]
    
    # Sort by angular distance (closest first)
    distances_within = angular_distances_deg[indices_within]
    sorted_indices = torch.argsort(distances_within)
    
    # Build results table
    results = []
    for rank, idx in enumerate(sorted_indices.cpu().numpy(), 1):
        token_id = indices_within[idx].item()
        angular_dist = distances_within[idx].item()
        cosine_sim = cosine_sims[token_id].item()
        
        # Decode token
        token_str = tokenizer.decode([token_id])
        
        results.append({
            'Rank': rank,
            'Token_ID': token_id,
            'Dist_deg': f"{angular_dist:.4f}",
            'Cosine': f"{cosine_sim:.6f}",
            'Token': repr(token_str)
        })
    
    results_df = pd.DataFrame(results)
    
    # Display (limit to first 100 rows)
    display_limit = min(100, len(results_df))
    print(f"\nShowing first {display_limit} of {len(results_df)} tokens:\n")
    print(results_df.head(display_limit).to_string(index=False))
    
    if len(results_df) > display_limit:
        print(f"\n... ({len(results_df) - display_limit} more tokens not shown)")
    
    # Summary
    print(f"\n" + "="*60)
    print("SUMMARY")
    print("="*60)
    print(f"Total tokens: {len(results_df):,}")


Showing first 100 of 111 tokens:

 Rank  Token_ID Dist_deg   Cosine                Token
    1    122401   0.1170 0.999998                  '垯'
    2    150523   0.1494 0.999997                  'ᐖ'
    3    150878   0.1520 0.999996                  '넖'
    4    151048   0.1769 0.999995                  '첧'
    5    151296   0.1958 0.999994                  '𝇊'
    6    149796   0.2017 0.999994                  '쿙'
    7    138175   0.2131 0.999993          'Cumhurbaş'
    8    150273   0.2149 0.999993                  '낔'
    9    137568   0.2203 0.999993            ' ücrets'
   10    150288   0.2264 0.999992                  'ﳥ'
   11    151265   0.2825 0.999988                  '𑂄'
   12    123302   0.2860 0.999988                  '𡐓'
   13    151083   0.2907 0.999987                  '퍿'
   14    123330   0.2954 0.999987                  '𬒗'
   15    150907   0.3190 0.999985                  '땧'
   16    149112   0.3310 0.999983                  '텦'
   17    151114   0.3403 0.999

## Summary

Point at locations on the sky map and identify tokens within an angular radius.

**Usage:**
1. Read coordinates from 04.1a sky map
2. Set `TARGET_LON` and `TARGET_LAT_CDF` (CDF-flattened values)
3. Set `ANGULAR_RADIUS` for field of view
4. Run to see tokens sorted by angular distance

Token strings are in the rightmost column for clean table alignment.